## Import Libraries needed

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  48.57 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  29.97 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  36.89 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  40.44 MB/s
Libraries imported.


## Download data from the web and convert to a Pandas Dataframe using BeautifulSoup

In [2]:
from bs4 import BeautifulSoup

data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(data.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


### Make the 1st row headers

In [3]:
headers = df.loc[0]
df = df[1:]
df.columns = headers
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### Drop rows with Borough not assigned

In [4]:
df[df['Borough'] != 'Not assigned'].count()

0
Postcode         212
Borough          212
Neighbourhood    212
dtype: int64

In [5]:
df = df[df['Borough'] != 'Not assigned']
df.shape

(212, 3)

### Merge Duplicate rows with same Borough into one

In [6]:
df = df.groupby('Postcode').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join}).reset_index()
df.head()

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


### Replace Not assigned neighborhood with Borough Name

In [7]:
df[df['Neighbourhood'] == 'Not assigned']

,Postcode,Neighbourhood,Borough
85,M7A,Not assigned,Queen's Park


In [8]:
df['Neighbourhood'] = df['Neighbourhood'].apply(lambda x: df.Borough if x == 'Not assigned' else df.Neighbourhood)
df[df['Neighbourhood'] == 'Not assigned']

,Postcode,Neighbourhood,Borough


In [9]:
df.shape

(103, 3)

### Get Latitude and Longitude data for all the boroughs

In [10]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py35_0   conda-forge
    ratelim:    0.1.6-py35_0 conda-forge

orderedset-2.0 100% |################################| Time: 0:00:00  58.68 MB/s
ratelim-0.1.6- 100% |################################| Time: 0:00:00  12.03 MB/s
geocoder-1.38. 100% |################################| Time: 0:00:00  42.16 MB/s


In [ ]:
df['Latitude'] = 0
df['Longitude'] = 0


for i in range(df.shape[0]):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(df['Postcode'][i]))
        lat_lng_coords = g.latlng
    df['Latitude'][i] = lat_lng_coords[0]
    df['Longitude'][i]= lat_lng_coords[1]
df

### Since the above method did not work and was taking a long time I had to interrupt the kernel. Will export the latitude and longitude from csv

In [11]:
loc = "http://cocl.us/Geospatial_data"
df1 = pd.read_csv(loc)
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_lat = pd.merge(df,df1,left_on = 'Postcode',right_on = 'Postal Code',how = 'left')
df_lat = df_lat.drop('Postal Code',axis = 1)
df_lat.head()

,Postcode,Neighbourhood,Borough,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353
1,M1C,"Rouge, Malvern",Scarborough,43.784535,-79.160497
2,M1E,"Rouge, Malvern",Scarborough,43.763573,-79.188711
3,M1G,"Rouge, Malvern",Scarborough,43.770992,-79.216917
4,M1H,"Rouge, Malvern",Scarborough,43.773136,-79.239476


### Use geopy library to get latitude and longitude of Toronto

In [15]:
address = 'Toronto,ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### The map of Toronto with neighborhoods

In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_lat['Latitude'], df_lat['Longitude'], df_lat['Borough'], df_lat['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Let us work with only Boroughs that contain the word Tornoto

In [17]:
df_toronto = df_lat[df_lat['Borough'].str.contains('Toronto')]
df_toronto.Borough.unique()

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

### let's visualize the Boroughs that contain Toronto word

In [18]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare credentials

In [19]:
CLIENT_ID = 'UAVWDKCJOIFQMEH3XWOYJSGQCWGH0UQZRNUSGZ1Q4UPDZ0RL' # your Foursquare ID
CLIENT_SECRET = 'H1BDGSIFI1HDPT4EWSKDF5XRIO5AHZODM2B333HY12RR0DKU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50

### Function to get nearby venues for all postal codes in Toronto

In [20]:
def getNearbyVenues(codes, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for code, lat, lng in zip(codes, latitudes, longitudes):
        print(code)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postalcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the function on all Postal Codes in Toronto

In [21]:
toronto_venues = getNearbyVenues(codes=df_toronto['Postcode'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7Y


In [22]:
toronto_venues.head()

,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,M4E,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


#### Check how many venues were returned for each code

In [23]:
toronto_group = toronto_venues.groupby('Postalcode').count()
toronto_group

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postalcode,,,,,,
M4E,4,4,4,4,4,4
M4K,42,42,42,42,42,42
M4L,19,19,19,19,19,19
M4M,39,39,39,39,39,39
M4N,4,4,4,4,4,4
M4P,8,8,8,8,8,8
M4R,19,19,19,19,19,19
M4S,36,36,36,36,36,36
M4T,4,4,4,4,4,4


#### Analyze each neighborhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Postalcode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postcode,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,

#### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped

,Postcode,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.00,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00000

### Function to sort venues in descending order

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for i in range(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1, indicators[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for i in range(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[i, 1:] = return_most_common_venues(toronto_grouped.iloc[i, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,Neighborhood,Health Food Store,Coffee Shop,Pub,Yoga Studio
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant
2,M4L,Park,Pub,Brewery,Burger Joint,Liquor Store
3,M4M,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant
4,M4N,Bus Line,Park,Swim School,Dim Sum Restaurant,Yoga Studio


## Run K means Clustering algorithm

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 1, 1, 1, 0, 1, 1, 1, 2, 1], dtype=int32)

#### Create a dataframe that contains cluster as well as the data

In [29]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')

toronto_merged.head()

,Postcode,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,M4E,"Rouge, Malvern",East Toronto,43.676357,-79.293031,4,Neighborhood,Health Food Store,Coffee Shop,Pub,Yoga Studio
41,M4K,"Rouge, Malvern",East Toronto,43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant
42,M4L,"Rouge, Malvern",East Toronto,43.668999,-79.315572,1,Park,Pub,Brewery,Burger Joint,Liquor Store
43,M4M,"Rouge, Malvern",East Toronto,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant
44,M4N,"Rouge, Malvern",Central Toronto,43.728020,-79.388790,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Yoga Studio


### Visualize Clusters

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## On analyzing the clusters, most fall in Cluster 1

In [32]:
toronto_merged['Cluster Labels'].value_counts()

1    32
2     3
4     1
3     1
0     1
Name: Cluster Labels, dtype: int64

In [33]:
toronto_merged[toronto_merged['Cluster Labels'] == 1]

,Postcode,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
41,M4K,"Rouge, Malvern",East Toronto,43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant
42,M4L,"Rouge, Malvern",East Toronto,43.668999,-79.315572,1,Park,Pub,Brewery,Burger Joint,Liquor Store
43,M4M,"Rouge, Malvern",East Toronto,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant
45,M4P,"Rouge, Malvern",Central Toronto,43.712751,-79.390197,1,Grocery Store,Gym,Park,Breakfast Spot,Hotel
46,M4R,"Rouge, Malvern",Central Toronto,43.715383,-79.405678,1,Coffee Shop,Sporting Goods Shop,Yoga Studio,Fast Food Restaurant,Clothing Store
47,M4S,"Rouge, Malvern",Central Toronto,43.704324,-79.388790,1,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Sushi Restaurant
49,M4V,"Rouge, Malvern",Central Toronto,43.686412,-79.400049,1,Coffee Shop,Pub,Supermarket,Sports Bar,Pizza Place
51,M4X,"Rouge, Malvern",Downtown Toronto,43.667967,-79.367675,1,Coffee Shop,Restaurant,Pizza Place,Market,Italian Restaurant
52,M4Y,"Rouge, Malvern",Downtown Toronto,43.665860,-79.383160,1,Burger Joint,Coffee Shop,Gastropub,Gay Bar,Japanese Restaurant
53,M5A,"Rouge, Malvern",Downtown Toronto,43.654260,-79.360636,1,Coffee Shop,Bakery,Café,Park,Mexican Restaurant


## Cluster 1 has Coffee shops and Restaurants as the most common venues...Let us analyze the other clusters

In [34]:
toronto_merged[toronto_merged['Cluster Labels'] == 0]

,Postcode,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
44,M4N,"Rouge, Malvern",Central Toronto,43.72802,-79.38879,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Yoga Studio


## Cluster 0 has only 1 postcode which looks like a mix of everything

In [35]:
toronto_merged[toronto_merged['Cluster Labels'] == 2]

,Postcode,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
48,M4T,"Rouge, Malvern",Central Toronto,43.689574,-79.383160,2,Playground,Park,Trail,Tennis Court,Deli / Bodega
50,M4W,"Rouge, Malvern",Downtown Toronto,43.679563,-79.377529,2,Park,Playground,Trail,Department Store,Falafel Restaurant
64,M5P,"Rouge, Malvern",Central Toronto,43.696948,-79.411307,2,Park,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio


## Cluster 2 has a lot of parks and trails

In [36]:
toronto_merged[toronto_merged['Cluster Labels'] == 3]

,Postcode,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
63,M5N,"Rouge, Malvern",Central Toronto,43.711695,-79.416936,3,Garden,Home Service,Department Store,Falafel Restaurant,Event Space


## Cluster 3 also has only 1 post code

In [37]:
toronto_merged[toronto_merged['Cluster Labels'] == 4]

,Postcode,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,M4E,"Rouge, Malvern",East Toronto,43.676357,-79.293031,4,Neighborhood,Health Food Store,Coffee Shop,Pub,Yoga Studio


### Cluster 4 again has only 1 post code with a mix of everything

### Let's split Cluster 1 again into 5 clusters and see if we can find any differences there

In [38]:
# create a dataframe with only C1 postcodes
toronto_C1 = pd.merge(toronto_grouped,toronto_merged[['Postcode','Cluster Labels']],left_on = 'Postcode',right_on = 'Postcode',how = 'left')
toronto_C1 = toronto_C1[toronto_C1['Cluster Labels']== 1]
toronto_C1_clustering = toronto_C1.drop(['Postcode','Cluster Labels'],1)
toronto_C1_clustering.head()

,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023810,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023810,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.000,0.023810,0.02381,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.02381,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.095238,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.02381,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.023810,0.0,0.023810,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.02381,0.02381,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.238095,0.02381,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.071429,0.02381,0.0,0.047619,0.0,0.0,0.0,0.0,0.023810,0.0,0.0,0.000000,0.0,0.023810,0.02381,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.023810,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023810,0.0,0.0,0.0,0.000000,0.02381,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0

In [39]:
# run k-means clustering
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_C1_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 4, 1, 1, 1, 1, 1, 1], dtype=int32)

In [40]:
toronto_C1 = toronto_C1.drop('Cluster Labels',1) #drop the old cluster labels
toronto_C1.insert(0, 'Cluster Labels', kmeans.labels_) #insert new cluster labels
toronto_merged_C1 = toronto_merged[toronto_merged['Cluster Labels']==1]
toronto_merged_C1 = toronto_merged_C1.drop('Cluster Labels',1) #again drop old cluster labels
toronto_merged_C1 = pd.merge(toronto_merged_C1, toronto_C1,on='Postcode',how = 'left')
toronto_merged_C1 = toronto_merged_C1.iloc[:,0:11]
toronto_merged_C1.head()

,Postcode,Neighbourhood,Borough,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
0,M4K,"Rouge, Malvern",East Toronto,43.679557,-79.352188,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,1
1,M4L,"Rouge, Malvern",East Toronto,43.668999,-79.315572,Park,Pub,Brewery,Burger Joint,Liquor Store,1
2,M4M,"Rouge, Malvern",East Toronto,43.659526,-79.340923,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,1
3,M4P,"Rouge, Malvern",Central Toronto,43.712751,-79.390197,Grocery Store,Gym,Park,Breakfast Spot,Hotel,4
4,M4R,"Rouge, Malvern",Central Toronto,43.715383,-79.405678,Coffee Shop,Sporting Goods Shop,Yoga Studio,Fast Food Restaurant,Clothing Store,1


## Cluster 0 has a lot of Coffee Shops, Deli etc.

In [41]:
toronto_merged_C1[toronto_merged_C1['Cluster Labels'] == 0]

,Postcode,Neighbourhood,Borough,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
10,M5B,"Rouge, Malvern",Downtown Toronto,43.657162,-79.378937,Clothing Store,Café,Coffee Shop,Ramen Restaurant,Middle Eastern Restaurant,0
11,M5C,"Rouge, Malvern",Downtown Toronto,43.651494,-79.375418,Coffee Shop,Gastropub,Hotel,Italian Restaurant,Restaurant,0
12,M5E,"Rouge, Malvern",Downtown Toronto,43.644771,-79.373306,Coffee Shop,Cocktail Bar,Restaurant,Farmers Market,Bakery,0
14,M5H,"Rouge, Malvern",Downtown Toronto,43.650571,-79.384568,Steakhouse,American Restaurant,Café,Hotel,Bar,0
16,M5K,"Rouge, Malvern",Downtown Toronto,43.647177,-79.381576,Coffee Shop,Café,Restaurant,Gastropub,Deli / Bodega,0
17,M5L,"Rouge, Malvern",Downtown Toronto,43.648198,-79.379817,Hotel,Café,Coffee Shop,American Restaurant,Deli / Bodega,0
22,M5W,"Rouge, Malvern",Downtown Toronto,43.646435,-79.374846,Cocktail Bar,Pub,Hotel,Restaurant,Café,0
23,M5X,"Rouge, Malvern",Downtown Toronto,43.648429,-79.382280,Coffee Shop,Café,Hotel,Deli / Bodega,Gastropub,0


## Cluster 1 also has a lot of coffee shops & restaurants with a mix of different cuisines like Italian, Japanese, Sushi, Thai etc.

In [42]:
toronto_merged_C1[toronto_merged_C1['Cluster Labels'] == 1]

,Postcode,Neighbourhood,Borough,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
0,M4K,"Rouge, Malvern",East Toronto,43.679557,-79.352188,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,1
1,M4L,"Rouge, Malvern",East Toronto,43.668999,-79.315572,Park,Pub,Brewery,Burger Joint,Liquor Store,1
2,M4M,"Rouge, Malvern",East Toronto,43.659526,-79.340923,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,1
4,M4R,"Rouge, Malvern",Central Toronto,43.715383,-79.405678,Coffee Shop,Sporting Goods Shop,Yoga Studio,Fast Food Restaurant,Clothing Store,1
5,M4S,"Rouge, Malvern",Central Toronto,43.704324,-79.388790,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Sushi Restaurant,1
6,M4V,"Rouge, Malvern",Central Toronto,43.686412,-79.400049,Coffee Shop,Pub,Supermarket,Sports Bar,Pizza Place,1
7,M4X,"Rouge, Malvern",Downtown Toronto,43.667967,-79.367675,Coffee Shop,Restaurant,Pizza Place,Market,Italian Restaurant,1
8,M4Y,"Rouge, Malvern",Downtown Toronto,43.665860,-79.383160,Burger Joint,Coffee Shop,Gastropub,Gay Bar,Japanese Restaurant,1
9,M5A,"Rouge, Malvern",Downtown Toronto,43.654260,-79.360636,Coffee Shop,Bakery,Café,Park,Mexican Restaurant,1
13,M5G,"Rouge, Malvern",Downtown Toronto,43.657952,-79.387383,Coffee Shop,Café,Thai Restaurant,Bubble Tea Shop,Burger Joint,1


### Cluster 2 has just one postal code with Grocery, Park, Nightclub, diner

In [43]:
toronto_merged_C1[toronto_merged_C1['Cluster Labels'] == 2]

,Postcode,Neighbourhood,Borough,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
24,M6G,"Rouge, Malvern",Downtown Toronto,43.669542,-79.422564,Grocery Store,Café,Park,Nightclub,Diner,2


### Cluster 3 is the airport area

In [44]:
toronto_merged_C1[toronto_merged_C1['Cluster Labels'] == 3]

,Postcode,Neighbourhood,Borough,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
21,M5V,"Rouge, Malvern",Downtown Toronto,43.628947,-79.39442,Airport Terminal,Airport Lounge,Airport Service,Harbor / Marina,Boat or Ferry,3


### CLuster 4 has Grocery, Hotel, park, Gym, Breakfast...maybe offices nearby

In [45]:
toronto_merged_C1[toronto_merged_C1['Cluster Labels'] == 4]

,Postcode,Neighbourhood,Borough,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
3,M4P,"Rouge, Malvern",Central Toronto,43.712751,-79.390197,Grocery Store,Gym,Park,Breakfast Spot,Hotel,4


In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_C1['Latitude'], toronto_merged_C1['Longitude'], toronto_merged_C1['Postcode'], toronto_merged_C1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters